In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re, string
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

import nltk
nltk.download('stopwords')

from nltk import word_tokenize
nltk.download('punkt')

from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

from pymystem3 import Mystem
from string import punctuation

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def preprocess_data(df_1):

  # Preparation
  def remove_stickers(text):
    if text != text:
      return np.nan
    # Паттерн для поиска стикеров в формате [стикер]
    pattern = r"[^\w.!?,:;$\s/\"\']*"
    cleaned_text = re.sub(pattern, "", str(text))
    return cleaned_text

  def remove_punctuation(text):
      return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

  def remove_numbers(text):
      return ''.join([i if not i.isdigit() else ' ' for i in text])

  def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

  mystem = Mystem()
  russian_stopwords = stopwords.words("russian")
  russian_stopwords.extend(['…', '«', '»', '...'])
  def lemmatize_text(text):
      tokens = mystem.lemmatize(text.lower())
      tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
      text = " ".join(tokens)
      return text

  russian_stopwords = stopwords.words("russian")
  russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
  def remove_stop_words(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

  def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
    return " ".join(tokens)

  # Removing rows with blank cells and NaN's
  df_1 = df_1.replace(r'^s*$', float('NaN'), regex = True)
  df_1 = df_1.dropna(subset=['Text'])

  # Shuffling
  df_1 = df_1.reset_index()
  df_1 = df_1.drop(['index'], axis=1)

  # Removing Stickers
  for i in range(df_1.shape[0]):
    df_1['Text'][i] = remove_stickers(str(df_1['Text'][i]))

  # Fast preproccessing
  preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))))
  df_1['Preproccessed'] = list(map(preproccessing, df_1['Text']))

  # Stemming
  stemmer = SnowballStemmer("russian")
  stemmed_texts_list = []
  for text in tqdm(df_1['Preproccessed']):
      tokens = word_tokenize(text)
      stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
      text = " ".join(stemmed_tokens)
      stemmed_texts_list.append(text)

  df_1['text_stem'] = stemmed_texts_list

  # Removing stop words
  sw_texts_list = []
  for text in tqdm(df_1['Preproccessed']):
      text = remove_stop_words(text)
      sw_texts_list.append(text)

  df_1['text_sw'] = sw_texts_list

  # Lemmatize
  lemm_texts_list = []
  for text in tqdm(df_1['text_sw']):
      try:
          text = lemmatize_text(text)
          lemm_texts_list.append(text)
      except Exception as e:
          print(e)

  df_1['text_lemm'] = lemm_texts_list

  return df_1

In [ ]:
def build_data_frame():
  data = pd.read_excel("Датасет.xlsx")

  df = pd.DataFrame()
  df.loc[:, 'responsibilities'] = data['responsibilities(Должностные обязанности)']
  df.loc[:, 'requirements'] = data['requirements(Требования к соискателю)']
  df.loc[:, 'terms'] = data['terms(Условия)']
  df.loc[:, 'notes'] = data['notes(Примечания)']

  for i in range(df.shape[0]):
    df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['requirements'][i]), "")
    df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['terms'][i]), "")
    df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['notes'][i]), "")


  df_1 = pd.DataFrame(columns = ["Text", "Class", "Preproccessed"])

  for x in df['responsibilities']:
    df_1 = df_1.append({"Text": x, "Class": "responsibilities", "Preproccessed": np.nan}, ignore_index=True)

  for x in df['requirements']:
    df_1 = df_1.append({"Text": x, "Class": "requirements", "Preproccessed": np.nan}, ignore_index=True)

  for x in df['terms']:
    df_1 = df_1.append({"Text": x, "Class": "terms", "Preproccessed": np.nan}, ignore_index=True)

  for x in df['notes']:
    df_1 = df_1.append({"Text": x, "Class": "notes", "Preproccessed": np.nan}, ignore_index=True)

  df_add = pd.read_csv("file_w.csv")
  #df_add_terms = pd.read_csv("file_terms.csv")
  #df_add_terms = df_add_terms.replace('Terms','terms')
  df_2 = pd.concat([df_1, df_add], ignore_index=True)

  return df_2

In [83]:
def train(print_accuracy=False):
  # Building data frame

  df_1 = build_data_frame()
  df_1 = preprocess_data(df_1)

  # Building model/pipeline

  #X = df_1['text_lemm']
  X = df_1['text_sw']
  y = df_1['Class']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
  classes = df_1['Class'].unique()

  logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=50)),
               ])
  logreg.fit(X_train, y_train)

  if print_accuracy:
    y_pred = logreg.predict(X_test)
    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred,target_names=classes))

  return logreg

In [84]:
import pickle

if __name__ == '__main__':
	model = train(print_accuracy=True)

	filename = 'model_v1.pk'
	with open(filename, 'wb') as file:
		pickle.dump(model, file)

<ipython-input-13-5b5d354fe18b>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Text'][i] = remove_stickers(str(df_1['Text'][i]))


  0%|          | 0/3537 [00:00<?, ?it/s]

  0%|          | 0/3537 [00:00<?, ?it/s]

  0%|          | 0/3537 [00:00<?, ?it/s]

accuracy 0.9171374764595104
                  precision    recall  f1-score   support

responsibilities       0.74      0.63      0.68        51
    requirements       0.97      0.88      0.92       333
           terms       0.91      0.97      0.94       423
           notes       0.90      0.93      0.92       255

        accuracy                           0.92      1062
       macro avg       0.88      0.85      0.86      1062
    weighted avg       0.92      0.92      0.92      1062



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
nb.fit(X_train, y_train)

In [ ]:
y_pred = nb.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=classes))



# Linear Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
sgd.fit(X_train, y_train)

In [ ]:
y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=25)),
               ])

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

# **Функции для подготовки текста к классификации**
# Подготовка текста
# Классификация текста
# Полученире данных в формате категория - текст

In [ ]:
def prepare_text(text):
  delimiters = "условия", "требования", "примечания", "обязанности", ";", "\n"
  regex_pattern = '|'.join(map(re.escape, delimiters))
  regex_pattern += "|" + '(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'
  slices =  re.split(regex_pattern, text.lower())

  result = []
  abbreviation_pattern = re.compile(r'\b\w+\.$')
  for sentence in slices:
    sentence = re.sub("[-|:]", " ", sentence)
    sentence = sentence.strip().capitalize()
    if result and re.match(abbreviation_pattern, sentence):
        result[-1] += ' ' + sentence
    else:
        result.append(sentence)

  out = []
  for i in result:
    i = re.sub("[-|:]"," ",i)
    i = i.strip()
    # i = remove_multiple_spaces(remove_numbers(remove_punctuation(i.lower())))
    # i = remove_stop_words(i)
    # i = lemmatize_text(i)
    if i == "":
      continue
    out.append(i)

  return out

In [ ]:
def categorize_text(text, model):
  slices = prepare_text(text)

  vacancy = {
      'responsibilities': [],
      'requirements': [],
      'terms': [],
      'notes': []
  }

  for text in slices:
    pred = model.predict([text])
    vacancy[pred[0]].append(text)
  return vacancy

In [ ]:
def get_final_categories(text):
  final_categories = {}
  categories = categorize_text(text)
  for k, v in categories.items():
    final_category_text = "\n".join(v)
    final_categories[k] = final_category_text
  return final_categories

In [ ]:
text = "Для работы вахтовым методом во Владимирскую область на предприятие по производству автобусов требуются маляры на полуавтомат.  Требования:   - покраска металлов краскопультом и безвоздушным путем.  Вахта 60/30. Оформление по ТК РФ.  График работы 6 дней в неделю по 12 часов в день. Есть как дневные смены, так и ночные.  Стабильная оплата труда.  Проживание в г. Владимире. До предприятия возит служебный автобус.  - компенсация проезда до 2500 руб"
text1 = "Вахта в город Москва.  Обязанности: - армирование каркаса;  Требования: - опыт в строительстве приветствуется; - работа в бригаде;  Условия: - продолжительность вахты 60/30 (продление вахты возможно); - Официальное трудоустройство; - ЗП в срок и без задержек; - Авансирование дважды в месяц по 15 000 рублей, 15 и 30 числа; - Питание трехразовое за счет организации; - Выдача спецодежды и Сизов без вычета из заработной платы; - Организованные отправки до объекта (покупка билетов); - Помощь в прохождение медицинского осмотра; - Возможность получить квалификационные удостоверения; - Карьерный рост до бригадира/мастера;"
text2 = "В стабильную, развивающуюся компанию требуются Бетонщики-Арматурщики на строительные объекты в Москве  Рассмотрим кандидатов как с опытом работы, так и без него.  ВАХТОВЫЙ МЕТОД РАБОТЫ 60/30  З/п за месяц - 67 000 на руки  З/п за вахту - 134 000 на руки  Обязанности:  - Армирование, бетонирование;  - Монтаж опалубки.  Требования:  - Соблюдение трудовой и бытовой дисциплины;  Условия:  - ВАХТА! 60/30;  - График работы 7/0 по 11 часов  - Прямой работодатель, ОФИЦИАЛЬНОЕ трудоустройство по ТК РФ  - Выплата заработной платы БЕЗ ЗАДЕРЖЕК. Оплата по часовой ставке , выплачивается стабильно 2 раза в месяц ;  - Проживание за счет компании  - Питание за счет компании  - Спецодежда за счет компании  - Покупаем билеты на вахту/с вахты"
get_final_categories(text1)